In [2]:
import pandas as pd 

df = pd.read_csv('gs://trabalho-final-pdm/gold/tabelas_knn.csv', storage_options={"token": "google_default"})
df


,ANO_ELEICAO,NM_VOTAVEL,NR_VOTAVEL,QT_VOTOS,Nome_do_subdistrito,Situacao_setor_x,Proporcao_negros_pardos_indigenas,Proporcao_feminino,Proporcao_18_39,RENDA,NUM_PARTIDO,PARTIDO
0,2016,MARIO MIGUEL GHANNAM,19234,1,U.T.P. PARQUE AMAZONIA,1.0,0.540136,0.521088,0.236735,2469.07,0,Coligações
1,2016,MARIO MIGUEL GHANNAM,19234,3,U.T.P. COIMBRA,1.0,0.165548,0.536913,0.201342,2762.71,0,Coligações
2,2016,ELIAS VAZ DE ANDRADE,40001,3,U.T.P. PARQUE AMAZONIA,1.0,0.540136,0.521088,0.236735,2469.07,40,PSB
3,2016,ELIAS VAZ DE ANDRADE,40001,1,U.T.P. CAMPUS UNIVERSITARIO/CONJUNTO ITATIAIA,1.0,0.450839,0.503597,0.253397,3783.75,40,PSB
4,2016,ELIAS VAZ DE ANDRADE,40001,7,U.T.P. PEDRO LUDOVICO/BELA VISTA/JARDINS DAS E...,1.0,0.403614,0.565060,0.236145,4292.31,40,PSB
...,...,...,...,...,...,...,...,...,...,...,...,...
969443,2024,MARCOS ANTONIO DA SILVA,25456,1,NaN,NaN,NaN,NaN,NaN,NaN,25,PRD
969444,2024,MARCOS ANTONIO DA SILVA,25456,1,U.T.P. VILA REGINA/PARQUE INDUSTRIAL PAULISTA/...,1.0,0.568110,0.525871,0.241816,1335.67,25,PRD
969445,2024,MARCOS ANTONIO DA SILVA,25456,2,NaN,NaN,NaN,NaN,NaN,NaN,25,PRD
969446,2024,MARCOS ANTONIO DA SILVA,25456,1,NaN,NaN,NaN,NaN,NaN,NaN,25,PRD


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Codificar 'Nome_do_subdistrito'
label_encoder = LabelEncoder()
df['Nome_do_subdistrito_encoded'] = label_encoder.fit_transform(df['Nome_do_subdistrito'].fillna('UNKNOWN'))

# Passo 1: Remover valores nulos na coluna 'RENDA' e 'Situacao_setor_x'
df = df.dropna(subset=['RENDA', 'Situacao_setor_x'])

# Passo 2: Seleção de atributos
features = ['Proporcao_negros_pardos_indigenas', 'Proporcao_feminino', 
            'Proporcao_18_39', 'RENDA', 'Nome_do_subdistrito_encoded', 'Situacao_setor_x']
target = 'NUM_PARTIDO'

X = df[features]
y = df[target]

# Escalabilidade Min-Max Scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Passo 3: Balanceamento de classes (Oversampling e Undersampling)
# Separar classes majoritária e minoritárias
majority_class = df[df['NUM_PARTIDO'] == y.value_counts().idxmax()]
minority_classes = df[df['NUM_PARTIDO'] != y.value_counts().idxmax()]

# Oversampling das classes minoritárias
minority_classes_resampled = resample(
    minority_classes, replace=True, n_samples=len(majority_class), random_state=42
)

# Combinar dados balanceados
balanced_df = pd.concat([majority_class, minority_classes_resampled])

# Atualizar as features (X) e os rótulos (y)
X_balanced = balanced_df[features]
y_balanced = balanced_df['NUM_PARTIDO']

# Dividir em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X_balanced, y_balanced, test_size=0.2, random_state=42
)

# Treinar o modelo Random Forest
random_forest = RandomForestClassifier(random_state=42)
random_forest.fit(X_train, y_train)

# Predição no conjunto de teste e cálculo da acurácia
y_pred = random_forest.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")


Accuracy: 0.49819663959182475


In [7]:
!pip install xgboost
!pip install lightgbm
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 MB 6.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 39.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 128.8 MB/s eta 0:00:0000:0100:01


In [21]:
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

# Modelos
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "LightGBM": LGBMClassifier(random_state=42),
    "CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42)
}

# Armazenar os resultados de acurácia
results = {}

# Treinar e avaliar cada modelo
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy

results


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001558 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 287
[LightGBM] [Info] Number of data points in the train set: 136411, number of used features: 6
[LightGBM] [Info] Start training from score -0.692246
[LightGBM] [Info] Start training from score -3.858577
[LightGBM] [Info] Start training from score -3.910638
[LightGBM] [Info] Start training from score -4.309173
[LightGBM] [Info] Start training from score -3.635850
[LightGBM] [Info] Start training from score -2.960378
[LightGBM] [Info] Start training from score -5.490148
[LightGBM] [Info] Start training from score -3.998182
[LightGBM] [Info] Start training from score -8.016765
[LightGBM] [Info] Start training from score -3.451723
[LightGBM] [Info] Start training from score -4.330667
[LightGBM] [Info] Start training from score -3.685155


/opt/conda/miniconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'Decision Tree': 0.49819663959182475,
 'LightGBM': 0.49819663959182475,
 'CatBoost': 0.49819663959182475,
 'Logistic Regression': 0.49819663959182475}

In [ ]:
## KNN e DBSCAN
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, silhouette_score
import pandas as pd

# Codificar 'Nome_do_subdistrito'
label_encoder = LabelEncoder()
df['Nome_do_subdistrito_encoded'] = label_encoder.fit_transform(df['Nome_do_subdistrito'].fillna('UNKNOWN'))

# Passo 1: Remover valores nulos na coluna 'RENDA' e 'Situacao_setor_x'
df = df.dropna(subset=['RENDA', 'Situacao_setor_x'])

# Passo 2: Seleção de atributos
features = ['Proporcao_negros_pardos_indigenas', 'Proporcao_feminino', 
            'Proporcao_18_39', 'RENDA', 'Nome_do_subdistrito_encoded', 'Situacao_setor_x']
target = 'NUM_PARTIDO'

X = df[features]
y = df[target]

# Escalabilidade Min-Max Scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Passo 3: Balanceamento de classes (Oversampling e Undersampling)
# Separar classes majoritária e minoritárias
majority_class = df[df['NUM_PARTIDO'] == y.value_counts().idxmax()]
minority_classes = df[df['NUM_PARTIDO'] != y.value_counts().idxmax()]

# Oversampling das classes minoritárias
minority_classes_resampled = resample(
    minority_classes, replace=True, n_samples=len(majority_class), random_state=42
)

# Combinar dados balanceados
balanced_df = pd.concat([majority_class, minority_classes_resampled])

# Atualizar as features (X) e os rótulos (y)
X_balanced = balanced_df[features]
y_balanced = balanced_df['NUM_PARTIDO']

# Dividir em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X_balanced, y_balanced, test_size=0.2, random_state=42
)

# **Parte 1: KNN**
# Treinar o modelo KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Predição e acurácia do KNN
y_pred_knn = knn.predict(X_test)
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print(f"KNN Accuracy: {accuracy_knn}")

# **Parte 2: DBSCAN**
# DBSCAN não supervisionado, então usamos X_scaled
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan_labels = dbscan.fit_predict(X_scaled)

# Avaliar clusters usando Silhouette Score
if len(set(dbscan_labels)) > 1:  # Silhouette Score requer mais de um cluster
    silhouette_avg = silhouette_score(X_scaled, dbscan_labels)
    print(f"DBSCAN Silhouette Score: {silhouette_avg}")
else:
    print("DBSCAN não identificou clusters suficientes para calcular Silhouette Score.")


KNN Accuracy: 0.4974342433217019


In [4]:
# Reimportar bibliotecas necessárias após o reset
import pandas as pd
from sklearn.utils import resample


# Verificar o balanceamento inicial da variável alvo 'NUM_PARTIDO'
class_counts = df['NUM_PARTIDO'].value_counts()

# Identificar a classe minoritária
minority_class_size = class_counts.min()

# Separar as classes majoritárias e minoritárias
majority_classes = df[df['NUM_PARTIDO'].isin(class_counts[class_counts > minority_class_size].index)]
minority_classes = df[df['NUM_PARTIDO'].isin(class_counts[class_counts == minority_class_size].index)]

# Balancear o dataset por undersampling nas classes majoritárias
majority_classes_resampled = resample(
    majority_classes,
    replace=False,  # Sem substituição para undersampling
    n_samples=len(minority_classes),  # Número de amostras igual à classe minoritária
    random_state=42
)

# Combinar as classes resampleadas
balanced_df = pd.concat([majority_classes_resampled, minority_classes])

# Verificar o balanceamento após o ajuste
balanced_class_counts = balanced_df['NUM_PARTIDO'].value_counts()

# Retornar informações sobre o balanceamento
class_counts, balanced_class_counts


(NUM_PARTIDO
 0     132119
 15     88860
 22     52102
 27     50438
 13     43515
 25     40727
 45     40314
 55     40313
 36     39737
 40     38674
 44     37647
 70     36976
 28     36271
 10     35137
 77     33740
 11     32533
 20     30981
 43     25913
 12     22885
 33     22234
 23     21642
 35     16811
 50     14457
 65     13933
 30     12776
 18      6586
 80      1657
 21       470
 Name: count, dtype: int64,
 NUM_PARTIDO
 21    470
 0      76
 15     49
 55     27
 40     25
 22     24
 25     23
 43     19
 10     19
 13     18
 20     18
 36     16
 45     15
 70     15
 77     15
 27     14
 44     13
 11     12
 33     11
 12     11
 35     10
 28      9
 65      9
 50      8
 30      5
 18      4
 23      4
 80      1
 Name: count, dtype: int64)

In [10]:
# Salvar DataFrame diretamente no GCS
gcs_path = "gs://trabalho-final-pdm/gold/tabelas_knn_balanceado.csv"

# Escrever para o Cloud Storage usando storage_options
balanced_df.to_csv(gcs_path, index=False, storage_options={"token": "google_default"})

In [9]:
balanced_df

,ANO_ELEICAO,NM_VOTAVEL,NR_VOTAVEL,QT_VOTOS,Nome_do_subdistrito,Situacao_setor_x,Proporcao_negros_pardos_indigenas,Proporcao_feminino,Proporcao_18_39,RENDA,NUM_PARTIDO,PARTIDO
580947,2020,EDGAR DUARTE GOMES,35700,1,NaN,NaN,NaN,NaN,NaN,NaN,35,PMB
371298,2020,CARLOS ALBERTO DA SILVA,15273,1,U.T.P. VILA PEDROSO,1.0,0.634799,0.491396,0.260038,2381.53,15,MDB
321705,2020,DENICIO CELIO TRINDADE,15700,2,U.T.P. LESTE UNIVERSITARIO,1.0,0.406308,0.508349,0.257885,3846.17,15,MDB
765943,2024,FABIO GALVAO SILVA,22200,1,U.T.P. CEASA/ALDEIA DO VALE,1.0,0.158405,0.513390,0.156695,17065.38,22,PL
240947,2016,RENER BUENO MARINHO BILAC,43200,1,U.T.P. PEDRO LUDOVICO/BELA VISTA/JARDINS DAS E...,1.0,0.403614,0.565060,0.236145,4292.31,43,PV
...,...,...,...,...,...,...,...,...,...,...,...,...
532566,2020,PARTIDO COMUNISTA BRASILEIRO,21,1,NaN,NaN,NaN,NaN,NaN,NaN,21,PCB
532568,2020,PARTIDO COMUNISTA BRASILEIRO,21,1,NaN,NaN,NaN,NaN,NaN,NaN,21,PCB
532569,2020,PARTIDO COMUNISTA BRASILEIRO,21,1,U.T.P. PARQUE AMAZONIA,1.0,0.540136,0.521088,0.236735,2469.07,21,PCB
532570,2020,PARTIDO COMUNISTA BRASILEIRO,21,1,NaN,NaN,NaN,NaN,NaN,NaN,21,PCB
